In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [1]:
!pip


Usage:   
  pip <command> [options]

Commands:
  install                     Install packages.
  download                    Download packages.
  uninstall                   Uninstall packages.
  freeze                      Output installed packages in requirements format.
  inspect                     Inspect the python environment.
  list                        List installed packages.
  show                        Show information about installed packages.
  check                       Verify installed packages have compatible dependencies.
  config                      Manage local and global configuration.
  search                      Search PyPI for packages.
  cache                       Inspect and manage pip's wheel cache.
  index                       Inspect information available from package indexes.
  wheel                       Build wheels from your requirements.
  hash                        Compute hashes of package archives.
  completion                  A helper co

In [2]:
import os

# Optionally force tensorflow on CPU
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Table of Contents
1. [General Usage](#general-usage)
    1. [Basic Usage](#basic-usage)
    2. [Advanced Usage](#advanced-usage)
2. [Data Generation](#data-gen)
    1. [Model Synthesis](#model-synth)
        1. [Keras Synthesis](#keras-synth)
        2. [PyTorch Synthesis](#torch-synth)
    2. [Parallel Synthesis](#parallel-synth)
        1. [Randomly Generated Networks](#random-synth)
3. [Training Prediction Models](#train-models)
    1. [Parsing Datasets](#parse-data)
        1. [Reading from JSON](#read-json)
    2. [Training MLPs](#train-mlps)
        1. [Data Preprocessing](#mlp-data)
        2. [Building & Training](#fit-mlps)
    3. [Training Transformers](#train-transformers)
        1. [Data Preprocessing](#transformer-data)
        2. [Building & Training](#fit-transformers)
    4. [Finetuning (Optional)](#finetune)
        1. [Finetuning an MLP](#finetune-mlp)
        2. [Loading and Retraining](#load-tuner)
4. [Testing Prediction Models](#test-models)
    1. [Benchmark Networks](#benchmark-test)
    2. [Plots](#plots)
        1. [Box Plots](#box-plots)
        2. [Bar Plots](#bar-plots)

# 1. General Usage <a class="anchor" id="general-usage"></a>

## 1.1 Basic Usage <a class="anchor" id="basic-usage"></a>

In [3]:
from keras.layers import Input, Dense, Activation
import keras

# Example of a keras Model to predict
input_size = 16
inputs = Input(shape=(input_size,))
x = Dense(32, activation="relu")(inputs)
x = Dense(32, activation="relu")(x)
x = Dense(32, activation="relu")(x)
outputs = Dense(5, activation="softmax")(x)

model_to_predict = keras.Model(inputs=inputs, outputs=outputs, name="Jet Classifier")
model_to_predict.build((None, input_size))

2024-09-01 18:47:37.676895: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-01 18:47:37.713414: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-01 18:47:37.713448: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-01 18:47:37.714366: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-01 18:47:37.739984: I tensorflow/core/platform/cpu_feature_guar

In [4]:
from rule4ml.models.estimators import MultiModelEstimator

# Load default estimator
estimator = MultiModelEstimator()
estimator.load_default_models()

In [5]:
MultiModelEstimator?

Init signature: MultiModelEstimator()
Docstring:      _summary_
File:           ~/anaconda3/envs/relu4ml_hls4mlofficial/lib/python3.11/site-packages/rule4ml/models/estimators.py
Type:           type
Subclasses:     

In [6]:
# MultiModelEstimator predictions are formatted as a DataFrame
prediction_df = estimator.predict(model_to_predict)

# each row is unique in the groupby, mean() is only called to convert DataFrameGroupBy into a nicely organized DataFrame
if not prediction_df.empty:
    prediction_df = prediction_df.groupby(
        ["Model", "Board", "Strategy", "Precision", "Reuse Factor"], observed=True
    ).mean()

prediction_df

BRAM (%)  \
Model          Board      Strategy Precision       Reuse Factor             
Jet Classifier pynq-z2    Latency  ap_fixed<2, 1>  1                 2.77   
                                                   2                 2.75   
                                                   4                 2.70   
                                                   8                 2.97   
                                                   16                2.97   
                                                   32                2.26   
                                                   64                0.83   
                                   ap_fixed<8, 3>  1                 2.63   
                                                   2                 2.63   
                                                   4                 2.59   
                                                   8                 2.76   
                                                   16                3.42   
                                                   32                2.99   
                                                   64                0.56   
                                   ap_fixed<16, 6> 1                 1.78   
                                                   2                 2.30   
                                                   4                 2.38   
                                                   8                 1.48   
                                                   16                2.90   
                                                   32                4.43   
                                                   64                0.75   
                          Resource ap_fixed<2, 1>  1                 5.67   
                                                   2                34.49   
                                                   4                13.99   
                                                   8                10.20   
                                                   16                7.84   
                                                   32                4.20   
                                                   64                3.44   
                                   ap_fixed<8, 3>  1                10.05   
                                                   2                55.80   
                                                   4                55.67   
                                                   8                20.88   
                                                   16               11.99   
                                                   32                5.06   
                                                   64                3.01   
                                   ap_fixed<16, 6> 1                11.69   
                                                   2               119.10   
                                                   4                89.59   
                                                   8                47.71   
                                                   16               27.98   
                                                   32                9.69   
                                                   64                5.45   
               zcu102     Latency  ap_fixed<2, 1>  1                 0.71   
                                                   2                 0.71   
                                                   4                 0.75   
                                                   8                 0.84   
                                                   16                0.81   
                                                   32                0.38   
                                                   64                0.54   
                                   ap_fixed<8, 3>  1                 0.57   
                                                   2         

In [ ]:
#prediction_df.to_html("keras_example.html")

# prediction_df.to_latex("keras_example.tex")
# prediction_df.to_csv("keras_example.csv")
# prediction_df.to_json("keras_example.json")
# prediction_df.to_xml("keras_example.xml")

## 1.2 Advanced Usage <a class="anchor" id="advanced-usage"></a>

In [3]:
import itertools

import keras
from keras.layers import Input, Dense
import torch

models_to_predict = []


# Example of a subclassed PyTorch model
class MyTopQuarks(torch.nn.Module):
    def __init__(self):
        super(MyTopQuarks, self).__init__()

        self.dense1 = torch.nn.Linear(10, 32)
        self.relu = torch.nn.ReLU()
        self.dense2 = torch.nn.Linear(32, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, inputs):
        x = self.dense1(inputs)
        x = self.relu(x)
        x = self.dense2(x)
        outputs = self.sigmoid(x)

        return outputs


models_to_predict.append(MyTopQuarks())

# Example of a keras Sequential model
input_size = 16
model_to_predict = keras.Sequential(
    layers=[
        keras.layers.Input(shape=(input_size,)),
        keras.layers.Dense(32, use_bias=True),
        keras.layers.Activation("relu"),
        keras.layers.Dense(32, use_bias=True),
        keras.layers.Activation("relu"),
        keras.layers.Dense(32, use_bias=True),
        keras.layers.Activation("relu"),
        keras.layers.Dense(5, use_bias=True),
        keras.layers.Activation("softmax"),
    ],
    name="Jet Classifier",
)
model_to_predict.build((None, input_size))

models_to_predict.append(model_to_predict)

hls_configs = [
    {
        "model": {
            "precision": "ap_fixed<8, 3>",
            "reuse_factor": 32,
            "strategy": strategy,
        },
        "board": board,
    }
    for board, strategy in itertools.product(["pynq-z2", "zcu102"], ["Latency", "Resource"])
]

2024-09-01 18:55:43.715958: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-01 18:55:43.761319: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-01 18:55:43.761353: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-01 18:55:43.762423: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-01 18:55:43.769610: I tensorflow/core/platform/cpu_feature_guar

In [4]:
from rule4ml.models.estimators import ModelWrapper

lut_model_wrapper = ModelWrapper()
lut_model_wrapper.load("./models/best_LUT_MLP_config.json", "./models/best_LUT_MLP.weights.h5")

lut_model_wrapper.predict(models_to_predict, hls_configs)

array([[39.422424 ],
       [51.593323 ],
       [ 3.6865003],
       [ 6.612974 ],
       [94.71447  ],
       [72.222435 ],
       [25.216333 ],
       [14.825675 ]], dtype=float32)

In [5]:
from rule4ml.models.estimators import MultiModelEstimator

estimator = MultiModelEstimator()
estimator.add_model_wrapper(lut_model_wrapper)

estimator.predict(models_to_predict, hls_configs)

,Model,Board,Strategy,Precision,Reuse Factor,LUT (%)
0,MyTopQuarks,pynq-z2,Latency,"ap_fixed<8, 3>",32,39.42
1,MyTopQuarks,pynq-z2,Resource,"ap_fixed<8, 3>",32,51.59
2,MyTopQuarks,zcu102,Latency,"ap_fixed<8, 3>",32,3.69
3,MyTopQuarks,zcu102,Resource,"ap_fixed<8, 3>",32,6.61
4,Jet Classifier,pynq-z2,Latency,"ap_fixed<8, 3>",32,94.71
5,Jet Classifier,pynq-z2,Resource,"ap_fixed<8, 3>",32,72.22
6,Jet Classifier,zcu102,Latency,"ap_fixed<8, 3>",32,25.22
7,Jet Classifier,zcu102,Resource,"ap_fixed<8, 3>",32,14.83


# 2. Data Generation <a class="anchor" id="data-gen"></a>

In [1]:
import os
import sys

# Specify Vivado path
#os.environ["PATH"] = "/opt/Xilinx/Vivado/2019.1/bin:" + os.environ["PATH"]
#os.environ["PATH"]  = "/home/share/Xilinx/Vivado/2023.2/bin" + os.environ["PATH"]
os.environ["PATH"]  = "/home/share/Xilinx/Vitis_HLS/2023.2/bin/" + os.environ["PATH"]

base_path = os.path.join(os.getcwd(), "..", "data_gen")
sys.path.append(base_path)

## 2.1 Model Synthesis <a class="anchor" id="model-synth"></a>

### 2.1.1 Keras Model <a class="anchor" id="keras-synth"></a>

In [2]:
import keras
from keras.layers import Input, Dense

from data_gen.nn_synth import synthesize_keras_model

input_size = 16
inputs = Input(shape=(input_size,))
x = Dense(32, activation="relu")(inputs)
x = Dense(32, activation="relu")(x)
x = Dense(32, activation="relu")(x)
outputs = Dense(5, activation="softmax")(x)

model_to_synthesize = keras.Model(inputs=inputs, outputs=outputs, name="Jet Classifier")
model_to_synthesize.build((None, input_size))

synthesis_result = synthesize_keras_model(
    model_to_synthesize,
    board="pynq-z2",
    strategy="Resource",
    precision="ap_fixed<8, 3>",
    reuse_factor=32,
    clock_period="10",
    io_type="io_parallel",
    project_dir="./hls4ml_prj",
    synth_uuid=None,
    verbose=0,
)

2024-09-01 20:41:03.416783: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-01 20:41:03.460998: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-01 20:41:03.461029: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-01 20:41:03.462117: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-01 20:41:03.469062: I tensorflow/core/platform/cpu_feature_guar

Interpreting Model
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 16]], output shape: [None, 16]
Layer name: dense, layer type: Dense, input shapes: [[None, 16]], output shape: [None, 32]
Layer name: dense_1, layer type: Dense, input shapes: [[None, 32]], output shape: [None, 32]
Layer name: dense_2, layer type: Dense, input shapes: [[None, 32]], output shape: [None, 32]
Layer name: dense_3, layer type: Dense, input shapes: [[None, 32]], output shape: [None, 5]
Interpreting Model
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 16]], output shape: [None, 16]
Layer name: dense, layer type: Dense, input shapes: [[None, 16]], output shape: [None, 32]
Layer name: dense_1, layer type: Dense, input shapes: [[None, 32]], output shape: [None, 32]
Layer name: dense_2, layer type: Dense, input shapes: [[None, 32]], output shape: [None, 32]
Layer name: dense_3, layer type: Dense, input shapes: [[None, 32]], output shape: [None, 5]
Creating

In [3]:
from data_gen.utils import save_to_json

save_to_json(synthesis_result, "./synthesis_result.json")

In [4]:
synthesis_result

{'meta_data': {'uuid': '58e57381-6726-4f1a-aa4a-29dc8b813a6e',
  'synthesis_start': '20240901-204110',
  'synthesis_end': '20240901-204132',
  'cpu': {'brand': '13th Gen Intel(R) Core(TM) i9-13900',
   'architecture': 'X86_64',
   'base_frequency': '3.8000 GHz',
   'logical_count': 32,
   'physical_count': 24}},
 'model_config': [{'class_name': 'InputLayer',
   'input_shape': ((None, 16),),
   'output_shape': ((None, 16),),
   'parameters': 0,
   'trainable_parameters': 0,
   'dtype': 'float32',
   'reuse_factor': 32},
  {'class_name': 'Dense',
   'input_shape': (None, 16),
   'output_shape': (None, 32),
   'parameters': 544,
   'trainable_parameters': 544,
   'neurons': 32,
   'use_bias': True,
   'dtype': 'float32',
   'reuse_factor': 32},
  {'class_name': 'Activation',
   'input_shape': (None, 32),
   'output_shape': (None, 32),
   'activation': 'relu',
   'parameters': 0,
   'trainable_parameters': 0,
   'dtype': 'float32',
   'reuse_factor': 32},
  {'class_name': 'Dense',
   'inpu

### 2.1.2 PyTorch Model <a class="anchor" id="torch-synth"></a>

In [ ]:
import torch

from data_gen.nn_synth import synthesize_torch_model
from data_gen.utils import save_to_json

model_to_synthesize = torch.nn.Sequential(
    torch.nn.Linear(10, 32),
    torch.nn.ReLU(),
    torch.nn.Linear(32, 1),
    torch.nn.Sigmoid(),
)

synthesis_result = synthesize_torch_model(
    model_to_synthesize,
    board="zcu102",
    strategy="Latency",
    precision="ap_fixed<8, 3>",
    reuse_factor=32,
    clock_period="10",
    io_type="io_parallel",
    project_dir="./hls4ml_prj",
    synth_uuid=None,
    verbose=0,
)

save_to_json(synthesis_result, "./synthesis_result.json")

## 2.2 Parallel Synthesis <a class="anchor" id="parallel-synth"></a>

### 2.2.1 Randomly Generated Networks <a class="anchor" id="random-synth"></a>

In [ ]:
from multiprocessing import Pool
import time

from data_gen.nn_gen import GeneratorSettings, generate_fc_network
from data_gen.nn_synth import (
    SynthSettings,
    synthesize_keras_model,
    parallel_generative_synthesis,
)

from data_gen.utils import IntRange, Power2Range, save_to_json

gen_settings = GeneratorSettings(
    input_range=Power2Range(16, 32),
    layer_range=IntRange(2, 3),
    neuron_range=Power2Range(16, 32),
    output_range=IntRange(1, 20),
    activations=["relu"],
)
synth_settings = SynthSettings(
    reuse_range=Power2Range(32, 64),
    precisions=["ap_fixed<2, 1>", "ap_fixed<8, 3>"],
    strategies=["Resource"],
)

n_procs = 3
with Pool(n_procs) as p:
    result = p.map_async(
        parallel_generative_synthesis,
        [
            {
                "job_id": f"{proc}",
                "n_models": 10,
                "project_dir": "./projects",
                "prj_overwrite": False,
                "save_path": "./",
                "rng_seed": 0,
                "gen_function": generate_fc_network,  # Keras only currently
                "gen_settings": gen_settings,
                "synth_function": synthesize_keras_model,
                "synth_settings": synth_settings,
            }
            for proc in range(1, n_procs + 1)
        ],
    )
    while not result.ready():
        time.sleep(1)
    result = result.get()
    p.terminate()
    p.join()

# 3. Training Prediction Models <a class="anchor" id="train-models"></a>

## 3.1 Parsing Datasets

### 3.1.1 Reading from JSON

In [ ]:
from rule4ml.parsers.data_parser import (
    read_from_json,
    ParsedDataFilter,
    get_global_data,
    get_sequential_data,
    to_dataframe,
)

from rule4ml.parsers.data_parser import (
    default_board_map,
    default_strategy_map,
    default_layer_type_map,
)

data_filter = ParsedDataFilter(
    max_output_size=200,
)

base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
json_data = read_from_json(
    os.path.join(base_path, "datasets/fcnn_dataset_15000.json"),
    data_filter,
)

meta_data, global_inputs, targets = get_global_data(json_data)
sequential_inputs = get_sequential_data(json_data)

# Ordinal encoding of categorical inputs
global_categorical_maps = {
    "strategy": default_strategy_map,
    "board": default_board_map,
}
sequential_categorical_maps = {
    "layer_type": default_layer_type_map,
}

df = to_dataframe(
    meta_data=meta_data,
    global_inputs=global_inputs,
    sequential_inputs=sequential_inputs,
    global_categorical_maps=global_categorical_maps,
    sequential_categorical_maps=sequential_categorical_maps,
    targets=targets,
)
df.head()

In [ ]:
df["sequential_inputs"].iloc[0]

In [ ]:
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
import numpy as np

seed_num = 1337
np.random.seed(seed_num)
keras.utils.set_random_seed(seed_num)
tf.config.experimental.enable_op_determinism()

train_df, test_df = train_test_split(df, test_size=0.05, random_state=seed_num)
print(f"Train Dataframe: {train_df.shape}")
print(f"Test Dataframe: {test_df.shape}")

## 3.2 Training MLPs <a class="anchor" id="train-mlps"></a>

### 3.2.1 Data Preprocessing <a class="anchor" id="mlp-data"></a>

In [ ]:
feature_labels = [  # Selecting input features
    "strategy",
    "board",
    # "precision",
    "bit_width",
    # "integer_bits",
    # "fractional_bits",
    "reuse_mean",
    # "dense_count",
    # "batchnormalization_count",
    # "add_count",
    # "concatenate_count",
    # "dropout_count",
    # "relu_count",
    # "sigmoid_count",
    # "tanh_count",
    # "softmax_count",
    # "dense_parameters_mean",
    # "dense_inputs_mean",
    # "dense_outputs_mean",
    # "dense_reuse_mean",
    "dense_inputs_mean",
    # "dense_inputs_min",
    # "dense_inputs_min_idx",
    # "dense_inputs_max",
    # "dense_inputs_max_idx",
    "dense_outputs_mean",
    # "dense_outputs_min",
    # "dense_outputs_min_idx",
    # "dense_outputs_max",
    # "dense_outputs_max_idx",
    "dense_parameters_mean",
    # "dense_parameters_min",
    # "dense_parameters_min_idx",
    # "dense_parameters_max",
    # "dense_parameters_max_idx",
    "dense_reuse_mean",
    # "dense_reuse_min",
    # "dense_reuse_min_idx",
    # "dense_reuse_max",
    # "dense_reuse_max_idx",
    "dense_count",
    "batchnormalization_inputs_mean",
    # "batchnormalization_inputs_min",
    # "batchnormalization_inputs_min_idx",
    # "batchnormalization_inputs_max",
    # "batchnormalization_inputs_max_idx",
    "batchnormalization_outputs_mean",
    # "batchnormalization_outputs_min",
    # "batchnormalization_outputs_min_idx",
    # "batchnormalization_outputs_max",
    # "batchnormalization_outputs_max_idx",
    "batchnormalization_parameters_mean",
    # "batchnormalization_parameters_min",
    # "batchnormalization_parameters_min_idx",
    # "batchnormalization_parameters_max",
    # "batchnormalization_parameters_max_idx",
    "batchnormalization_count",
    # "add_inputs_mean",
    # "add_inputs_min",
    # "add_inputs_min_idx",
    # "add_inputs_max",
    # "add_inputs_max_idx",
    # "add_outputs_mean",
    # "add_outputs_min",
    # "add_outputs_min_idx",
    # "add_outputs_max",
    # "add_outputs_max_idx",
    "add_count",
    # "concatenate_inputs_mean",
    # "concatenate_inputs_min",
    # "concatenate_inputs_min_idx",
    # "concatenate_inputs_max",
    # "concatenate_inputs_max_idx",
    # "concatenate_outputs_mean",
    # "concatenate_outputs_min",
    # "concatenate_outputs_min_idx",
    # "concatenate_outputs_max",
    # "concatenate_outputs_max_idx",
    "concatenate_count",
    # "dropout_inputs_mean",
    # "dropout_inputs_min",
    # "dropout_inputs_min_idx",
    # "dropout_inputs_max",
    # "dropout_inputs_max_idx",
    # "dropout_outputs_mean",
    # "dropout_outputs_min",
    # "dropout_outputs_min_idx",
    # "dropout_outputs_max",
    # "dropout_outputs_max_idx",
    "dropout_count",
    # "relu_inputs_mean",
    # "relu_inputs_min",
    # "relu_inputs_min_idx",
    # "relu_inputs_max",
    # "relu_inputs_max_idx",
    # "relu_outputs_mean",
    # "relu_outputs_min",
    # "relu_outputs_min_idx",
    # "relu_outputs_max",
    # "relu_outputs_max_idx",
    "relu_count",
    # "sigmoid_inputs_mean",
    # "sigmoid_inputs_min",
    # "sigmoid_inputs_min_idx",
    # "sigmoid_inputs_max",
    # "sigmoid_inputs_max_idx",
    # "sigmoid_outputs_mean",
    # "sigmoid_outputs_min",
    # "sigmoid_outputs_min_idx",
    # "sigmoid_outputs_max",
    # "sigmoid_outputs_max_idx",
    "sigmoid_count",
    # "tanh_inputs_mean",
    # "tanh_inputs_min",
    # "tanh_inputs_min_idx",
    # "tanh_inputs_max",
    # "tanh_inputs_max_idx",
    # "tanh_outputs_mean",
    # "tanh_outputs_min",
    # "tanh_outputs_min_idx",
    # "tanh_outputs_max",
    # "tanh_outputs_max_idx",
    "tanh_count",
    "softmax_inputs_mean",
    # "softmax_inputs_min",
    # "softmax_inputs_min_idx",
    # "softmax_inputs_max",
    # "softmax_inputs_max_idx",
    "softmax_outputs_mean",
    # "softmax_outputs_min",
    # "softmax_outputs_min_idx",
    # "softmax_outputs_max",
    # "softmax_outputs_max_idx",
    "softmax_count",
    # "total_mult",
    # "total_add",
    # "total_logical",
    # "total_lookup",
]

train_inputs_df = train_df[feature_labels].copy()
test_inputs_df = test_df[feature_labels].copy()

train_inputs_df.head()

In [ ]:
target_labels = ["lut"]

train_targets_df = train_df[target_labels].copy()
test_targets_df = test_df[target_labels].copy()

train_targets_df.head()

### 3.2.2 Building and Training <a class="anchor" id="fit-mlps"></a>

In [ ]:
from rule4ml.models.estimators import MLPSettings, ModelWrapper

input_shape = (None, len(train_inputs_df.columns))
output_shape = (None, len(train_targets_df.columns))

bram_mlp_settings = MLPSettings(
    embedding_outputs=[16 for _ in range(len(global_categorical_maps))],
    numerical_dense_layers=[32, 16, 32],
    dense_layers=[256, 256, 256, 64, 32, 64, 64],
    dense_dropouts=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
)
dsp_mlp_settings = MLPSettings(
    embedding_outputs=[16 for _ in range(len(global_categorical_maps))],
    numerical_dense_layers=[64, 32, 32],
    dense_layers=[256, 16, 32, 32, 64],
    dense_dropouts=[0.0, 0.0, 0.0, 0.0, 0.0],
)
ff_mlp_settings = MLPSettings(
    embedding_outputs=[16 for _ in range(len(global_categorical_maps))],
    numerical_dense_layers=[64, 16, 32],
    dense_layers=[64, 128, 64, 256, 32],
    dense_dropouts=[0.0, 0.0, 0.0, 0.0, 0.0],
)
lut_mlp_settings = MLPSettings(
    embedding_outputs=[16 for _ in range(len(global_categorical_maps))],
    numerical_dense_layers=[64, 16, 32, 32],
    dense_layers=[64, 128, 128, 64],
    dense_dropouts=[0.0, 0.0, 0.0, 0.0],
)
cycles_mlp_settings = MLPSettings(
    embedding_outputs=[16 for _ in range(len(global_categorical_maps))],
    numerical_dense_layers=[32, 16, 64],
    dense_layers=[256, 32, 32, 32, 256, 128, 128, 32, 16, 16, 64],
    dense_dropouts=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
)

mlp_settings = lut_mlp_settings

model_wrapper = ModelWrapper()
model_wrapper.build_mlp_model(
    mlp_settings,
    input_shape=input_shape,
    output_shape=output_shape,
    categorical_maps=global_categorical_maps,
    model_name=f"{'-'.join([x.upper() for x in target_labels])}_MLP",
    verbose=1,
)

In [ ]:
from rule4ml.models.estimators import TrainSettings
from rule4ml.models.metrics import parametric_smape, parametric_r2

smape = parametric_smape(0, "-".join([x.upper() for x in target_labels]))
r2 = parametric_r2(0, "-".join([x.upper() for x in target_labels]))

bram_train_settings = TrainSettings(
    num_epochs=50,
    batch_size=64,
    learning_rate=1e-4,
    loss_function="mae",
    metrics=[smape, r2],
)
dsp_train_settings = TrainSettings(
    num_epochs=50,
    batch_size=32,
    learning_rate=1e-4,
    loss_function="mae",
    metrics=[smape, r2],
)
ff_train_settings = TrainSettings(
    num_epochs=50,
    batch_size=64,
    learning_rate=1e-4,
    loss_function="mae",
    metrics=[smape, r2],
)
lut_train_settings = TrainSettings(
    num_epochs=50,
    batch_size=32,
    learning_rate=1e-4,
    loss_function="mae",
    metrics=[smape, r2],
)
cycles_train_settings = TrainSettings(
    num_epochs=50,
    batch_size=64,
    learning_rate=1e-3,
    loss_function="mae",
    metrics=[smape, r2],
)

train_settings = cycles_train_settings

model_wrapper.build_dataset(
    train_inputs_df,
    train_targets_df,
    train_settings.batch_size,
    val_ratio=0.15,
    train_repeats=10,
    shuffle=True,
)

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, TensorBoard

from datetime import datetime

start_time = datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = os.path.join("./logs", f"{model_wrapper.model.name}_{start_time}")
checkpoint_dir = os.path.join("./checkpoints", f"{model_wrapper.model.name}_{start_time}")
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
checkpoint_file = os.path.join(checkpoint_dir, f"{'-'.join(target_labels)}_best.weights.h5")

tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=True,
    write_images=True,
    write_steps_per_second=False,
    update_freq="epoch",
    embeddings_freq=1,
)

checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_file,
    save_weights_only=True,
    monitor="val_loss",
    mode="min",
    save_best_only=True,
)


def scheduler(epoch, lr):
    if (epoch + 1) % 20 == 0:
        return lr * np.exp(-0.2)
    return lr


lr_callback = LearningRateScheduler(scheduler)

callbacks = [
    tensorboard_callback,
    checkpoint_callback,
    # lr_callback
]

fit_history = model_wrapper.fit(train_settings, callbacks=callbacks, verbose=1)

## 3.3 Training Transformers <a class="anchor" id="train-transformers"></a>

### 3.3.1 Data Preprocessing <a class="anchor" id="transformer-data"></a>

In [ ]:
global_feature_labels = [
    "strategy",
    "board",
    "bit_width",
    "reuse_mean",
    "dense_inputs_mean",
    "dense_outputs_mean",
    "dense_parameters_mean",
    "dense_reuse_mean",
    "dense_count",
    "batchnormalization_inputs_mean",
    "batchnormalization_outputs_mean",
    "batchnormalization_parameters_mean",
    "batchnormalization_count",
    "add_count",
    "concatenate_count",
    "dropout_count",
    "relu_count",
    "sigmoid_count",
    "tanh_count",
    "softmax_inputs_mean",
    "softmax_outputs_mean",
    "softmax_count",
]
sequential_feature_labels = [
    "layer_type",
    "layer_input_size",
    "layer_output_size",
    "layer_parameter_count",
    "layer_reuse",
]

feature_labels = global_feature_labels
if len(sequential_feature_labels) > 0:
    feature_labels += ["sequential_inputs"]
inputs_df = df[feature_labels].copy()
inputs_df["sequential_inputs"] = inputs_df["sequential_inputs"].apply(
    lambda x: x[sequential_feature_labels]
)

inputs_df.head()

In [ ]:
inputs_df["sequential_inputs"].iloc[0]

In [ ]:
target_labels = ["lut"]
targets_df = df[target_labels].copy()
targets_df.head()

### 3.3.2 Building and Training <a class="anchor" id="fit-transformers"></a>

In [ ]:
from rule4ml.models.estimators import TransformerSettings, ModelWrapper

global_input_shape = (None, len(inputs_df.columns) - 1)  # not considering "sequential_inputs"
sequential_input_shape = (None, len(inputs_df["sequential_inputs"].iloc[0].columns))
output_shape = (None, len(targets_df.columns))

model_wrapper = ModelWrapper()
model_wrapper.build_transformer_model(
    TransformerSettings(
        global_dense_layers=[128, 192, 192],
        seq_dense_layers=[32, 64, 96],
        global_numerical_dense_layers=[16, 8],
        seq_numerical_dense_layers=[32],
        num_blocks=1,
        num_heads=8,
        ff_dim=256,
        output_dim=192,
        dropout_rate=0.2,
        embedding_outputs=[24, 24, 16, 8],
        dense_layers=[192, 128, 64, 32, 64, 128, 256, 32],
        dense_dropouts=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    ),
    global_input_shape=global_input_shape,
    sequential_input_shape=sequential_input_shape,
    output_shape=output_shape,
    global_categorical_maps=global_categorical_maps,
    sequential_categorical_maps=sequential_categorical_maps,
    model_name=f"{'-'.join([x.upper() for x in target_labels])}_Transformer",
    verbose=1,
)

In [ ]:
from rule4ml.models.estimators import TrainSettings

train_settings = TrainSettings(num_epochs=50)
model_wrapper.build_dataset(
    inputs_df,
    targets_df,
    train_settings.batch_size,
    val_ratio=0.15,
    train_repeats=1,
    shuffle=True,
)

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, TensorBoard

from datetime import datetime

start_time = datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = os.path.join("./logs", f"{model_wrapper.model.name}_{start_time}")
checkpoint_dir = os.path.join("./checkpoints", f"{model_wrapper.model.name}_{start_time}")
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
checkpoint_file = os.path.join(checkpoint_dir, f"{'-'.join(target_labels)}_best.weights.h5")

tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=True,
    write_images=True,
    write_steps_per_second=False,
    update_freq="epoch",
    embeddings_freq=1,
)

checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_file,
    save_weights_only=True,
    monitor="val_loss",
    mode="min",
    save_best_only=True,
)


def scheduler(epoch, lr):
    if (epoch + 1) % 20 == 0:
        return lr * np.exp(-0.2)
    return lr


lr_callback = LearningRateScheduler(scheduler)

callbacks = [
    tensorboard_callback,
    checkpoint_callback,
    # lr_callback
]

fit_history = model_wrapper.fit(train_settings, callbacks=callbacks, verbose=1)

## 3.4 Finetuning (Optional) <a class="anchor" id="finetune"></a>

### 3.4.1 Finetuning an MLP <a class="anchor" id="finetune-mlp"></a>

In [ ]:
from rule4ml.models.tuning import Searcher
from rule4ml.models.estimators import ModelWrapper

target_labels = ["lut"]

train_targets_df = train_df[target_labels].copy()
test_targets_df = test_df[target_labels].copy()

model_wrapper = ModelWrapper()
searcher = Searcher(model_wrapper)
searcher.mlp_search(
    train_inputs_df,
    train_targets_df,
    global_categorical_maps,
    directory="./mlp_search",
    verbose=1,
)
searcher.tuner.results_summary()

### 3.4.2 Loading and Retraining <a class="anchor" id="load-tuner"></a>

In [ ]:
from rule4ml.models.tuning import Searcher
from rule4ml.models.estimators import ModelWrapper

model_wrapper = ModelWrapper()
searcher = Searcher(model_wrapper)
searcher.load_tuner(
    train_inputs_df,
    train_targets_df,
    global_categorical_maps,
    "./mlp_search",
    "20240715-090815",
)
searcher.tuner.results_summary()

In [ ]:
from rule4ml.models.estimators import TrainSettings

model_wrapper = searcher.model_wrapper
model_wrapper.fit(searcher.train_settings, verbose=1)

# 4. Testing Prediction Models <a class="anchor" id="test-models"></a>

## 4.1 Benchmark Networks <a class="anchor" id="benchmark-test"></a>

In [ ]:
import keras
from keras.layers import (
    Dense,
    Add,
    Input,
    BatchNormalization,
    Conv2D,
    Flatten,
)


def get_test_model(name):
    model = None
    if name == "jet":
        input_size = 16
        inputs = Input(shape=(input_size,))
        x = Dense(32, use_bias=True)(inputs)
        x = Activation("relu")(x)
        x = Dense(32, use_bias=True)(x)
        x = Activation("relu")(x)
        x = Dense(32, use_bias=True)(x)
        x = Activation("relu")(x)
        x = Dense(5, use_bias=True)(x)
        outputs = Activation("softmax")(x)

        model = keras.Model(inputs=inputs, outputs=outputs, name=name)
        model.build([None, input_size])

    if name == "quarks":
        input_size = 10
        inputs = Input(shape=(input_size,))
        x = Dense(32, use_bias=True)(inputs)
        x = Activation("relu")(x)
        x = Dense(1, use_bias=True)(x)
        outputs = Activation("sigmoid")(x)

        model = keras.Model(inputs=inputs, outputs=outputs, name=name)
        model.build([None, input_size])

    if name == "anomaly":
        input_size = 128
        inputs = Input(shape=(input_size,))
        x = Dense(8, use_bias=True)(inputs)
        x = Activation("relu")(x)
        x = Dense(4, use_bias=True)(x)
        x = Activation("relu")(x)
        x = Dense(128, use_bias=True)(x)
        x = Activation("relu")(x)
        x = Dense(4, use_bias=True)(x)
        x = Activation("relu")(x)
        x = Dense(128, use_bias=True)(x)
        outputs = Activation("softmax")(x)

        model = keras.Model(inputs=inputs, outputs=outputs, name=name)
        model.build([None, input_size])

    if name == "bipc":
        input_size = 36
        inputs = Input(shape=(input_size,))
        x = Dense(36, use_bias=False)(inputs)

        y = Activation("relu")(x)
        for i in range(5):
            y = Dense(36, use_bias=False)(y)
            y = Add()([x, y])
            y = Activation("relu")(y)
        outputs = y

        model = keras.Model(inputs=inputs, outputs=outputs, name=name)
        model.build([None, input_size])

    if name == "cookie":
        input_size = 512
        inputs = Input(shape=(input_size,))
        x = Dense(4, use_bias=True)(inputs)
        x = Activation("relu")(x)
        x = Dense(32, use_bias=True)(x)
        x = Activation("relu")(x)
        x = Dense(32, use_bias=True)(x)
        x = Activation("relu")(x)
        x = Dense(5, use_bias=True)(x)
        outputs = Activation("softmax")(x)

        model = keras.Model(inputs=inputs, outputs=outputs, name=name)
        model.build([None, input_size])

    if name == "mnist":
        input_size = 784
        inputs = Input(shape=(input_size,))
        x = Dense(16, use_bias=True)(inputs)
        x = Activation("relu")(x)
        x = Dense(10, use_bias=True)(x)
        outputs = Activation("softmax")(x)

        model = keras.Model(inputs=inputs, outputs=outputs, name=name)
        model.build([None, input_size])

    if name == "automlp":
        input_size = 7
        inputs = Input(shape=(input_size,))
        x = Dense(12, use_bias=True)(inputs)
        x = Activation("relu")(x)
        x = Dense(16, use_bias=True)(x)
        x = Activation("relu")(x)
        x = Dense(12, use_bias=True)(x)
        x = Activation("relu")(x)
        x = Dense(2, use_bias=True)(x)
        outputs = Activation("softmax")(x)

        model = keras.Model(inputs=inputs, outputs=outputs, name=name)
        model.build([None, input_size])

    if name == "particle":
        input_size = 14
        inputs = Input(shape=(input_size,))
        x = Dense(32, use_bias=True)(inputs)
        x = Activation("relu")(x)
        x = Dense(32, use_bias=True)(x)
        x = Activation("relu")(x)
        x = Dense(32, use_bias=True)(x)
        x = Activation("relu")(x)
        x = Dense(3, use_bias=True)(x)
        outputs = Activation("softmax")(x)

        model = keras.Model(inputs=inputs, outputs=outputs, name=name)
        model.build([None, input_size])

    if name == "custom1":
        input_size = 16
        inputs = Input(shape=(input_size,))
        x = Dense(64, use_bias=True)(inputs)
        x = Activation("relu")(x)
        x = Dense(32, use_bias=True)(x)
        x = Activation("relu")(x)
        x = Dense(32, use_bias=True)(x)
        x = Activation("relu")(x)
        x = Dense(32, use_bias=True)(x)
        x = Activation("relu")(x)
        x = Dense(10, use_bias=True)(x)
        outputs = Activation("softmax")(x)

        model = keras.Model(inputs=inputs, outputs=outputs, name=name)
        model.build([None, input_size])

    if name == "custom2":
        input_size = 128
        inputs = Input(shape=(input_size,))
        x = Dense(16, use_bias=True)(inputs)
        x = Activation("relu")(x)
        x = Dense(64, use_bias=True)(x)
        x = Activation("relu")(x)
        x = Dense(32, use_bias=True)(x)
        x = Activation("relu")(x)
        x = Dense(64, use_bias=True)(x)
        x = Activation("relu")(x)
        x = Dense(32, use_bias=True)(x)
        x = Activation("relu")(x)
        x = Dense(50, use_bias=True)(x)
        outputs = Activation("softmax")(x)

        model = keras.Model(inputs=inputs, outputs=outputs, name=name)
        model.build([None, input_size])

    if name == "custom3":

        def residual_block(x, units):
            y = Dense(units)(x)
            y = BatchNormalization()(y)
            y = Activation("relu")(y)

            y = Dense(units)(y)
            y = BatchNormalization()(y)

            if x.shape[-1] == units:
                y = Add()([x, y])
            else:
                x = Dense(units)(x)
                x = BatchNormalization()(x)
                y = Add()([x, y])

            y = Activation("relu")(y)
            return y

        input_size = 64
        inputs = Input(shape=(input_size,))
        x = Dense(32, use_bias=True)(inputs)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)

        x = residual_block(x, units=32)
        x = residual_block(x, units=32)

        x = Dense(10)(x)
        outputs = Activation("softmax")(x)

        model = keras.Model(inputs=inputs, outputs=outputs, name=name)
        model.build([None, input_size])

    if name == "conv2d-nopool":
        input_size = (16, 16, 1)
        inputs = Input(input_size)
        x = Conv2D(16, (3, 3), padding="same")(inputs)
        x = Activation("relu")(x)
        x = Conv2D(4, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = Flatten()(x)
        x = Dense(2, use_bias=True)(x)
        outputs = Activation("softmax")(x)

        model = keras.Model(inputs=inputs, outputs=outputs, name=name)
        model.build([None, input_size])

    return model

In [ ]:
from rule4ml.models.estimators import ModelWrapper, MultiModelEstimator
import itertools

hls_configs = [
    {
        "model": {
            "precision": "ap_fixed<8, 3>",
            "reuse_factor": 32,
            "strategy": strategy,
            "bram_factor": 1000000000,
            "trace_output": False,
        },
        "clock_period": 10.0,
        "io_type": "io_parallel",
        "board": board,
    }
    for board, strategy in itertools.product(["pynq-z2", "zcu102"], ["Latency", "Resource"])
]

model_names = [
    "jet",
    "quarks",
    "anomaly",
    "bipc",
    "cookie",
    "mnist",
    "automlp",
    "particle",
    "custom1",
    "custom2",
    "custom3",
]
models = [get_test_model(name) for name in model_names]

target_labels = ["bram", "dsp", "ff", "lut", "cycles"]

estimator = MultiModelEstimator()
for label in target_labels:
    model_wrapper = ModelWrapper()
    model_wrapper.load(
        f"./models/best_{label.upper()}_MLP_config.json",
        f"./models/best_{label.upper()}_MLP.weights.h5",
    )
    estimator.add_model_wrapper(model_wrapper)

prediction_df = estimator.predict(models, hls_configs)

In [ ]:
prediction_df.sort_values(["Board", "Strategy", "Reuse Factor"]).round(0)

## 4.2 Plots <a class="anchor" id="plots"></a>

### 4.2.1 Box Plots <a class="anchor" id="box-plots"></a>

In [ ]:
import os
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
import numpy as np

from rule4ml.parsers.data_parser import (
    read_from_json,
    ParsedDataFilter,
    get_global_data,
    get_sequential_data,
    to_dataframe,
)

from rule4ml.parsers.data_parser import (
    default_board_map,
    default_strategy_map,
    default_layer_type_map,
)

data_filter = ParsedDataFilter(
    max_output_size=200,
)

base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
json_data = read_from_json(
    os.path.join(base_path, "datasets/fcnn_dataset_15000.json"),
    data_filter,
)

meta_data, global_inputs, targets = get_global_data(json_data)
sequential_inputs = get_sequential_data(json_data)

# Ordinal encoding of categorical inputs
global_categorical_maps = {
    "strategy": default_strategy_map,
    "board": default_board_map,
}
sequential_categorical_maps = {
    "layer_type": default_layer_type_map,
}

df = to_dataframe(
    meta_data=meta_data,
    global_inputs=global_inputs,
    sequential_inputs=sequential_inputs,
    global_categorical_maps=global_categorical_maps,
    sequential_categorical_maps=sequential_categorical_maps,
    targets=targets,
)

seed_num = 1337
np.random.seed(seed_num)
keras.utils.set_random_seed(seed_num)
tf.config.experimental.enable_op_determinism()

train_df, test_df = train_test_split(df, test_size=0.05, random_state=seed_num)

feature_labels = [
    "strategy",
    "board",
    "bit_width",
    "reuse_mean",
    "dense_inputs_mean",
    "dense_inputs_min",
    "dense_inputs_min_idx",
    "dense_inputs_max",
    "dense_inputs_max_idx",
    "dense_outputs_mean",
    "dense_outputs_min",
    "dense_outputs_min_idx",
    "dense_outputs_max",
    "dense_outputs_max_idx",
    "dense_parameters_mean",
    "dense_parameters_min",
    "dense_parameters_min_idx",
    "dense_parameters_max",
    "dense_parameters_max_idx",
    "dense_reuse_mean",
    "dense_reuse_min",
    "dense_reuse_min_idx",
    "dense_reuse_max",
    "dense_reuse_max_idx",
    "dense_count",
    "batchnormalization_inputs_mean",
    "batchnormalization_outputs_mean",
    "batchnormalization_parameters_mean",
    "batchnormalization_count",
    "add_count",
    "concatenate_count",
    "dropout_count",
    "relu_count",
    "sigmoid_count",
    "tanh_count",
    "softmax_inputs_mean",
    "softmax_outputs_mean",
    "softmax_count",
]

test_inputs_df = test_df[feature_labels].copy()
print(f"Test Inputs: {test_inputs_df.shape}")

In [ ]:
from rule4ml.models.estimators import ModelWrapper

prediction_labels = ["bram", "dsp", "ff", "lut", "cycles"]
test_targets_df = test_df[prediction_labels].copy()

wrappers = []
prediction_errors = []
for label in prediction_labels:
    wrapper = ModelWrapper()
    wrapper.load(
        f"./models/best_{label.upper()}_MLP_config.json",
        f"./models/best_{label.upper()}_MLP.weights.h5",
    )
    wrappers.append(wrapper)

    pred = wrapper.predict_from_df(test_inputs_df).squeeze()
    gn = test_targets_df[label].values

    prediction_errors.append(np.abs(gn - pred))

In [ ]:
np.asarray(prediction_errors).shape

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

plt.rcParams.update({"font.size": 16})

fig, axis = plt.subplots(2, 2, figsize=(12, 8), width_ratios=[3, 1])
axis = np.reshape(axis, -1)
fig.subplots_adjust(hspace=0.1, wspace=0.4)

flier_ax, box_ax = axis[0], axis[2]

iqr_weight = 1.5

resources_errors = prediction_errors[:4]
resources_labels = prediction_labels[:4]

threshold = 10.0
below_threshold = []
for errors in np.asarray(resources_errors):
    below_threshold.append(np.sum(errors < threshold) / len(errors))
print(f"Resources below {threshold}%: {below_threshold}")
print(f"Resources Mean: {np.mean(below_threshold)}")

bplot = box_ax.boxplot(
    resources_errors,
    whis=iqr_weight,
    tick_labels=[x.upper() for x in resources_labels],
    showfliers=True,
    showmeans=True,
    meanline=True,
    vert=True,
    patch_artist=True,
)
fliers = flier_ax.boxplot(
    resources_errors,
    whis=iqr_weight,
    tick_labels=["" for x in resources_labels],
    showfliers=True,
    showmeans=True,
    meanline=True,
    vert=True,
    patch_artist=True,
)

colors = ["pink", "yellow", "lightgreen", "lightblue", "FFA500"]
for patch, color in zip(bplot["boxes"], colors):
    patch.set_facecolor(color)

box_ax.set_ylim(-1, 30)
flier_ax.set_ylim(30, 200)

box_ax.yaxis.grid(True)
box_ax.spines.top.set_visible(False)
box_ax.xaxis.tick_bottom()
box_ax.set_yticks([0, 5, 10, 15, 20, 25, 30])

flier_ax.yaxis.grid(True)
flier_ax.spines.bottom.set_visible(False)
flier_ax.xaxis.tick_top()
flier_ax.set_yticks([30, 50, 75, 100, 125, 150, 175, 200])

d = 0.5
kwargs = dict(
    marker=[(-1, -d), (1, d)],
    markersize=12,
    linestyle="none",
    color="k",
    mec="k",
    mew=1,
    clip_on=False,
)
flier_ax.plot([0, 1], [0, 0], transform=flier_ax.transAxes, **kwargs)
box_ax.plot([0, 1], [1, 1], transform=box_ax.transAxes, **kwargs)

median_line = Line2D([0], [0], color="orange", linestyle="--", linewidth=1.5, label="Median")
mean_line = Line2D([0], [0], color="green", linestyle="--", linewidth=1.5, label="Mean")

handles = [median_line, mean_line]
labels = ["Median", "Mean"]

legends = fig.legend(
    handles,
    labels,
    bbox_to_anchor=[0.9, 1],
    # loc="upper left",
    loc="upper right",
    ncol=len(labels) // 2,
)

ytext = fig.text(0.06, 0.5, "Error (%)", va="center", rotation="vertical", size=18)
suptitle = fig.suptitle("Prediction Errors - Boxplots", fontsize=20, y=0.95)

latency_flier_ax, latency_box_ax = axis[1], axis[3]

iqr_weight = 1.5

latency_errors = [prediction_errors[4]]
latency_labels = [prediction_labels[4]]

threshold = 100.0
below_threshold = []
for errors in np.asarray(latency_errors):
    below_threshold.append(np.sum(errors < threshold) / len(errors))
print(f"Latency below {threshold} cycles: {below_threshold}")

latency_bplot = latency_box_ax.boxplot(
    latency_errors,
    whis=iqr_weight,
    widths=0.33,
    tick_labels=["Cycles"],
    showfliers=True,
    showmeans=True,
    meanline=True,
    vert=True,
    patch_artist=True,
)
latency_fliers = latency_flier_ax.boxplot(
    latency_errors,
    whis=iqr_weight,
    widths=0.33,
    tick_labels=["" for x in latency_labels],
    showfliers=True,
    showmeans=True,
    meanline=True,
    vert=True,
    patch_artist=True,
)

colors = ["lightblue"]
for patch, color in zip(latency_bplot["boxes"], colors):
    patch.set_facecolor(color)

latency_box_ax.set_ylim(-10, 200)
latency_flier_ax.set_ylim(200, 650)

latency_box_ax.yaxis.grid(True)
latency_box_ax.spines.top.set_visible(False)
latency_box_ax.xaxis.tick_bottom()
latency_box_ax.set_yticks(np.arange(0, 225, 25))

latency_flier_ax.yaxis.grid(True)
latency_flier_ax.spines.bottom.set_visible(False)
latency_flier_ax.xaxis.tick_top()
latency_flier_ax.set_yticks(np.arange(200, 700, 100))

d = 0.5
kwargs = dict(
    marker=[(-1, -d), (1, d)],
    markersize=12,
    linestyle="none",
    color="k",
    mec="k",
    mew=1,
    clip_on=False,
)
latency_flier_ax.plot([0, 1], [0, 0], transform=latency_flier_ax.transAxes, **kwargs)
latency_box_ax.plot([0, 1], [1, 1], transform=latency_box_ax.transAxes, **kwargs)

latency_ytext = fig.text(0.66, 0.5, "Error (Cycles)", va="center", rotation="vertical", size=18)

resource_caption = fig.text(0.355, 0.04, "(a)", va="center", size=18)
latency_caption = fig.text(0.808, 0.04, "(b)", va="center", size=18)

# fig.savefig(
#     "/mnt/c/Users/Y540/Desktop/box_plot_merged.jpg",
#     dpi=300,
#     bbox_extra_artists=(legends, ytext, suptitle, latency_ytext, resource_caption, latency_caption),
#     bbox_inches="tight",
# )
plt.show()

### 4.2.2 Bar Plots <a class="anchor" id="bar-plots"></a>

In [ ]:
from rule4ml.models.estimators import ModelWrapper, MultiModelEstimator
import numpy as np
import itertools

prediction_labels = ["bram", "dsp", "ff", "lut", "cycles"]

model_names = [
    "jet",
    "quarks",
    "anomaly",
    "bipc",
    "cookie",
    "mnist",
    "automlp",
    "particle",
    "custom1",
    "custom2",
    "custom3",
]
test_models = [get_test_model(name) for name in model_names]

hls_configs = [
    {
        "model": {
            "precision": precision,
            "reuse_factor": reuse,
            "strategy": strategy,
            "bram_factor": 1000000000,
            "trace_output": False,
        },
        "clock_period": 10.0,
        "io_type": "io_parallel",
        "board": board,
    }
    for board, strategy, precision, reuse in itertools.product(
        ["pynq-z2", "zcu102"],
        ["Latency", "Resource"],
        ["ap_fixed<2, 1>", "ap_fixed<8, 3>", "ap_fixed<16, 6>"],
        [1, 2, 4, 8, 16, 32, 64],
    )
]

estimator = MultiModelEstimator()
estimator.load_default_models()
predictions = []

prediction_df = estimator.predict(test_models, hls_configs)

In [ ]:
prediction_df["BRAM"] = prediction_df["BRAM"].apply(lambda x: min(x, 200.0))
prediction_df["DSP"] = prediction_df["DSP"].apply(lambda x: min(x, 200.0))
prediction_df["FF"] = prediction_df["FF"].apply(lambda x: min(x, 200.0))
prediction_df["LUT"] = prediction_df["LUT"].apply(lambda x: min(x, 200.0))

precision_order = ["ap_fixed<2, 1>", "ap_fixed<8, 3>", "ap_fixed<16, 6>"]
prediction_df["Precision"] = pd.Categorical(
    prediction_df["Precision"], categories=precision_order, ordered=True
)

prediction_df.head(14)

In [ ]:
from rule4ml.parsers.data_parser import (
    read_from_json,
    get_global_data,
    get_sequential_data,
    to_dataframe,
    default_strategy_map,
    default_board_map,
    default_layer_type_map,
)

benchmark_data = read_from_json("../datasets/benchmark_data.json")

benchmark_meta_data, benchmark_global_inputs, benchmark_targets = get_global_data(benchmark_data)
benchmark_sequential_inputs = get_sequential_data(benchmark_data)

global_categorical_maps = {
    "strategy": default_strategy_map,
    "board": default_board_map,
}
sequential_categorical_maps = {
    "layer_type": default_layer_type_map,
}

benchmark_df = to_dataframe(
    meta_data=benchmark_meta_data,
    global_inputs=benchmark_global_inputs,
    sequential_inputs=benchmark_sequential_inputs,
    global_categorical_maps={},
    sequential_categorical_maps={},
    targets=benchmark_targets,
)
benchmark_gn_df = benchmark_df[
    [
        "model_name",
        "board",
        "strategy",
        "precision",
        "global_reuse",
        "bram",
        "dsp",
        "ff",
        "lut",
        "cycles",
    ]
].copy()
benchmark_gn_df = benchmark_gn_df.rename(
    {
        "model_name": "Model",
        "board": "Board",
        "strategy": "Strategy",
        "precision": "Precision",
        "global_reuse": "Reuse Factor",
        "bram": "BRAM",
        "dsp": "DSP",
        "ff": "FF",
        "lut": "LUT",
        "cycles": "CYCLES",
    },
    axis=1,
)
benchmark_gn_df.loc[benchmark_gn_df["Strategy"] == "latency", "Strategy"] = "Latency"
benchmark_gn_df.loc[benchmark_gn_df["Strategy"] == "resource", "Strategy"] = "Resource"

benchmark_gn_df["BRAM"] = benchmark_gn_df["BRAM"].apply(lambda x: min(x, 200.0))
benchmark_gn_df["DSP"] = benchmark_gn_df["DSP"].apply(lambda x: min(x, 200.0))
benchmark_gn_df["FF"] = benchmark_gn_df["FF"].apply(lambda x: min(x, 200.0))
benchmark_gn_df["LUT"] = benchmark_gn_df["LUT"].apply(lambda x: min(x, 200.0))

precision_order = ["ap_fixed<2, 1>", "ap_fixed<8, 3>", "ap_fixed<16, 6>"]
benchmark_gn_df["Precision"] = pd.Categorical(
    benchmark_gn_df["Precision"], categories=precision_order, ordered=True
)

benchmark_gn_df.head(14)

In [ ]:
gn_grouped_mean = (
    benchmark_gn_df.groupby(["Strategy", "Board", "Precision", "Reuse Factor"])[
        [
            "BRAM",
            "DSP",
            "FF",
            "LUT",
            # "CYCLES"
        ]
    ]
    .mean()
    .round(0)
    .astype(int)
)

prediction_grouped_mean = (
    prediction_df.groupby(["Strategy", "Board", "Precision", "Reuse Factor"])[
        [
            "BRAM",
            "DSP",
            "FF",
            "LUT",
            # "CYCLES"
        ]
    ]
    .mean()
    .round(0)
    .astype(int)
)

In [ ]:
merged_df = pd.merge(
    gn_grouped_mean,
    prediction_grouped_mean,
    on=("Strategy", "Board", "Precision", "Reuse Factor"),
    suffixes=(" (G)", " (P)"),
)

merged_df = merged_df[
    [
        "BRAM (G)",
        "BRAM (P)",
        "DSP (G)",
        "DSP (P)",
        "FF (G)",
        "FF (P)",
        "LUT (G)",
        "LUT (P)",
        # "CYCLES (G)", "CYCLES (P)",
    ]
]
merged_df.head()

In [ ]:
import matplotlib.pyplot as plt
from rule4ml.parsers.utils import fixed_precision_to_bit_width

plt.rcParams.update({"font.size": 14})

grouped = merged_df.xs(("pynq-z2",), level=["Board"]).groupby(["Precision", "Strategy"])

n_groups = len(grouped)
n_cols = 2
n_rows = 3

fig, axes = plt.subplots(
    n_rows, n_cols, dpi=300, figsize=(16, 10), squeeze=False, sharex=True, sharey=False
)
axes = axes.flatten()

width = 0.11
colors = ["#008000", "#FF5964", "#17BEBB", "#FFA500"]
reuse_factors = prediction_df["Reuse Factor"].unique()
num_resources = 4
resource_gap = 0

total_width = num_resources * (2 * width + resource_gap) - resource_gap
start = np.arange(1, len(reuse_factors) + 1) - total_width / 2

row_idx = 0
col_idx = 0
for ax, ((precision, strategy), df) in zip(axes, grouped):
    for i, (col_gn, col_pred) in enumerate(zip(df.columns[::2], df.columns[1::2])):
        gn_vals = df[col_gn]
        pred_vals = df[col_pred]

        resource_indices = start + i * (2 * width + resource_gap)

        for j, reuse_factor in enumerate(reuse_factors):
            gn_label = ""
            pred_label = ""
            if j == 0:
                gn_label = f"{col_gn}"
                pred_label = f"{col_pred}"

            ax.bar(
                resource_indices[j] - width / 2,
                gn_vals[j],
                width,
                label=gn_label,
                color=colors[i % len(colors)],
                edgecolor="black",
            )
            ax.bar(
                resource_indices[j] + width / 2,
                pred_vals[j],
                width,
                label=pred_label,
                color=colors[i % len(colors)],
                edgecolor="black",
                hatch="///",
            )

    total_bits, fraction_bits = fixed_precision_to_bit_width(precision)

    ax.set_title(f"{strategy}, {total_bits}-bit width")
    ax.set_xticks(start + (num_resources - 1) * (width + resource_gap / 2))
    ax.set_xticklabels(reuse_factors, rotation=45)

    # if col_idx == 0:
    #     ax.set_ylabel("Utilization (%)")

    # if row_idx == n_rows - 1:
    #     ax.set_xlabel("Reuse Factor")

    col_idx += 1
    if col_idx == n_cols:
        row_idx += 1
        col_idx = 0

handles, labels = ax.get_legend_handles_labels()
legends = fig.legend(
    handles,
    labels,
    title="Resources",
    bbox_to_anchor=[0.3, 1.03],
    loc="upper left",
    # loc="upper right",
    ncol=len(labels) // 2,
)

xtext = fig.text(0.5, 0.035, "Reuse Factor", ha="center", size=18)
ytext = fig.text(0.07, 0.5, "Utilization (%)", va="center", rotation="vertical", size=18)

suptitle = fig.suptitle("Pynq-Z2: Resource Utilization Trends", fontsize=20, y=1.075)

plt.subplots_adjust(hspace=0.275, wspace=0.125)
plt.show()

# fig.savefig(
#     "/mnt/c/Users/Y540/Desktop/pynq_avg_bars.jpg",
#     dpi=300,
#     bbox_extra_artists=(legends, xtext, ytext, suptitle),
#     bbox_inches="tight",
# )

In [ ]:
plt.rcParams.update({"font.size": 14})

grouped = merged_df.xs(("zcu102",), level=["Board"]).groupby(["Precision", "Strategy"])

n_groups = len(grouped)
n_cols = 2
n_rows = 3

fig, axes = plt.subplots(
    n_rows, n_cols, dpi=300, figsize=(16, 10), squeeze=False, sharex=True, sharey=False
)
axes = axes.flatten()

width = 0.11
colors = ["#008000", "#FF5964", "#17BEBB", "#FFA500"]
reuse_factors = prediction_df["Reuse Factor"].unique()
num_resources = 4
resource_gap = 0

total_width = num_resources * (2 * width + resource_gap) - resource_gap
start = np.arange(1, len(reuse_factors) + 1) - total_width / 2

row_idx = 0
col_idx = 0
for ax, ((precision, strategy), df) in zip(axes, grouped):
    for i, (col_gn, col_pred) in enumerate(zip(df.columns[::2], df.columns[1::2])):
        gn_vals = df[col_gn]
        pred_vals = df[col_pred]

        resource_indices = start + i * (2 * width + resource_gap)

        for j, reuse_factor in enumerate(reuse_factors):
            gn_label = ""
            pred_label = ""
            if j == 0:
                gn_label = f"{col_gn}"
                pred_label = f"{col_pred}"

            ax.bar(
                resource_indices[j] - width / 2,
                gn_vals[j],
                width,
                label=gn_label,
                color=colors[i % len(colors)],
                edgecolor="black",
            )
            ax.bar(
                resource_indices[j] + width / 2,
                pred_vals[j],
                width,
                label=pred_label,
                color=colors[i % len(colors)],
                edgecolor="black",
                hatch="///",
            )

    total_bits, fraction_bits = fixed_precision_to_bit_width(precision)

    ax.set_title(f"{strategy}, {total_bits}-bit width")
    ax.set_xticks(start + (num_resources - 1) * (width + resource_gap / 2))
    ax.set_xticklabels(reuse_factors, rotation=45)

    col_idx += 1
    if col_idx == n_cols:
        row_idx += 1
        col_idx = 0

handles, labels = ax.get_legend_handles_labels()
legends = fig.legend(
    handles,
    labels,
    title="Resources",
    bbox_to_anchor=[0.3, 1.03],
    loc="upper left",
    ncol=len(labels) // 2,
)

xtext = fig.text(0.5, 0.035, "Reuse Factor", ha="center", size=18)
ytext = fig.text(0.07, 0.5, "Utilization (%)", va="center", rotation="vertical", size=18)

suptitle = fig.suptitle("ZCU102: Resource Utilization Trends", fontsize=20, y=1.075)

plt.subplots_adjust(hspace=0.275, wspace=0.125)
plt.show()

# fig.savefig(
#     "/mnt/c/Users/Y540/Desktop/zcu_avg_bars.jpg",
#     dpi=300,
#     bbox_extra_artists=(legends, xtext, ytext, suptitle),
#     bbox_inches="tight",
# )

In [ ]:
gn_grouped_mean = (
    benchmark_gn_df.groupby(["Strategy", "Board", "Precision", "Reuse Factor"])[["CYCLES"]]
    .mean()
    .round(0)
    .astype(int)
)

prediction_grouped_mean = (
    prediction_df.groupby(["Strategy", "Board", "Precision", "Reuse Factor"])[["CYCLES"]]
    .mean()
    .round(0)
    .astype(int)
)

merged_df = pd.merge(
    gn_grouped_mean,
    prediction_grouped_mean,
    on=("Strategy", "Board", "Precision", "Reuse Factor"),
    suffixes=(" (G)", " (P)"),
)

merged_df = merged_df[
    [
        "CYCLES (G)",
        "CYCLES (P)",
    ]
]
merged_df.head()

plt.rcParams.update({"font.size": 14})

grouped = merged_df.groupby(["Board", "Strategy", "Precision"])

n_groups = len(grouped)
n_cols = 3
n_rows = (n_groups // n_cols) + (n_groups % n_cols > 0)

fig, axes = plt.subplots(
    n_rows, n_cols, dpi=300, figsize=(16, 10), squeeze=False, sharex=True, sharey=False
)
axes = axes.flatten()

width = 0.35
colors = ["#17BEBB", "#FFA500"]
reuse_factors = prediction_df["Reuse Factor"].unique()
num_resources = 1
resource_gap = 0

total_width = num_resources * (2 * width + resource_gap) - resource_gap
start = np.arange(1, len(reuse_factors) + 1) - total_width / 2

row_idx = 0
col_idx = 0
for ax, ((board, strategy, precision), df) in zip(axes, grouped):
    for i, (col_gn, col_pred) in enumerate(zip(df.columns[::2], df.columns[1::2])):
        gn_vals = df[col_gn]
        pred_vals = df[col_pred]

        resource_indices = start + i * (2 * width + resource_gap)

        for j, reuse_factor in enumerate(reuse_factors):
            gn_label = ""
            pred_label = ""
            if j == 0:
                gn_label = f"{col_gn}"
                pred_label = f"{col_pred}"

            ax.bar(
                resource_indices[j] - width / 2,
                gn_vals[j],
                width,
                label=gn_label,
                color=colors[i % len(colors)],
                edgecolor="black",
            )
            ax.bar(
                resource_indices[j] + width / 2,
                pred_vals[j],
                width,
                label=pred_label,
                color=colors[i % len(colors) + 1],
                edgecolor="black",
                hatch="///",
            )

    total_bits, fraction_bits = fixed_precision_to_bit_width(precision)

    ax.set_title(f"{board}, {strategy}, {total_bits}-bit width")
    ax.set_xticks(start + (num_resources - 1) * (width + resource_gap / 2))
    ax.set_xticklabels(reuse_factors, rotation=45)

    col_idx += 1
    if col_idx == n_cols:
        row_idx += 1
        col_idx = 0

handles, labels = ax.get_legend_handles_labels()
legends = fig.legend(handles, labels, bbox_to_anchor=[0.8, 1], loc="upper left")

xtext = fig.text(0.5, 0.05, "Reuse Factor", ha="center", size=18)
ytext = fig.text(0.07, 0.5, "Cycles", va="center", rotation="vertical", size=18)

fig.suptitle("Clock Cycle Trends", fontsize=20)

plt.subplots_adjust(hspace=0.4, wspace=0.2)
plt.show()

# fig.savefig(
#     "/mnt/c/Users/Y540/Desktop/cycles_avg_bars.jpg",
#     dpi=300,
#     bbox_extra_artists=(legends, xtext, ytext),
#     bbox_inches="tight",
# )